# Machine Learning

Andrew Esch, Evan Lee, Collin Stratton

Dr. Isac Artzi

CST-425

4/17/2022

# Introduction


# Code


## Setup

In [ ]:
# import libraries
from sklearn.model_selection import train_test_split
import tensorflow as tf
import os.path
import numpy as np
import pandas as pd

from pathlib import Path

import requests
from PIL import Image
from io import BytesIO
import cv2

## Get Images and Label Data

In [ ]:
# import images
# image_dir = Path('C:/Users/Drew/OneDrive/Pictures/ml-food/images')
image_dir = Path('/Users/collinstratton/Documents/archive/images')
images_paths = list(image_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], images_paths))

images_paths = pd.Series(images_paths, name='Image_Path').astype(str)
labels = pd.Series(labels, name='Label')

images = pd.concat([images_paths, labels], axis=1)

In [ ]:
# split images into categories
category_samples = []
for category in images['Label'].unique():
    category_slice = images.query('Label == @category')
    category_samples.append(category_slice.sample(300, random_state=1))

# group images into tables and check the number of images by category
images_samples = pd.concat(category_samples, axis=0)
images_samples['Label'].value_counts()

## Preprocess Images

In [ ]:
# Testing and training
train_data, test_data = train_test_split(images_samples, train_size=0.7, shuffle=True, random_state=1)

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

train_images = train_generator.flow_from_dataframe(
    dataframe=train_data,
    x_col='Image_Path',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_data,
    x_col='Image_Path',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_data,
    x_col='Image_Path',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg',
)

# freeze the weights of the network to maintain transfer learning
pretrained_model.trainable = False

inputs = pretrained_model.input

# create two layers of our own
x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

# create output layer with the 101 classes
output_layer = tf.keras.layers.Dense(101, activation='softmax')(x)

# unite the original model with the new layers
model = tf.keras.Model(inputs, output_layer)

print(model.summary())

## Train Model

In [ ]:
# Training
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    ]
)

# Training Results - Prediction Accuracy
results = model.evaluate(test_images, verbose=1)
print("Precision: {:.2f}%".format(results[1] * 100))

## Image Prediction

In [ ]:
# make a prediction with an internet image
url = 'https://www.hola.com/imagenes/cocina/tecnicas-de-cocina/20210820194795/como-hacer-patatas-fritas-perfectas/0-986-565/portada-patatas-age-t.jpg'
response = requests.get(url)
img = Image.open(BytesIO(response.content))
img = np.array(img).astype(float) / 255

img = cv2.resize(img, (224, 224))
predict = model.predict(img.reshape(-1, 224, 224, 3))

print(np.argmax(predict))

# Conclusion

# Resources
